# Feature subset analysis

## Set-up

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import logging
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import TunedThresholdClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import(roc_auc_score,
                            matthews_corrcoef,
                            f1_score,
                            precision_recall_curve,
                            auc)
from imblearn.metrics import geometric_mean_score
from sklearn.utils import resample
import shap

In [ ]:
while not os.getcwd().endswith('chest-pain-dissertation'):
    os.chdir('../')

In [ ]:
os.getcwd()

## Define functions

In [ ]:
def load_csv(filepath: str) -> pd.DataFrame:

    try:
        df = pd.read_csv(filepath)
        print(f"Data loaded successfully: {filepath}")
        return df
    except Exception as e:
        print(f"Error loading {filepath}: {e}")
        raise(e)

In [ ]:
def load_model(filepath: str):

    try:
        model = joblib.load(filepath)
        print(f"Model loaded successfully: {filepath}")
        return model
    except Exception as e:
        msg = f"Error loading the model {filepath}: {e}"
        print(msg)
        raise(e)

In [ ]:
def get_model_name(pipe: Pipeline) -> str:
    return list(pipe.named_steps.keys())[-1]

In [ ]:
def map_model_name(model_name: str) -> str:

    model_name_dict = {
        'lreg_model': 'Logistic Regression',
        'rfc_model': 'Random Forest',
        'xgb_model': 'XGBoost',
        'lgbm_model': 'LightGBM',
    }
    try:
        return model_name_dict[model_name]
    except:
        msg = (f'{model_name} not found. Model name must be one of '
               f'{model_name_dict.keys()}')
        print(msg)
        raise ValueError(msg)

In [ ]:
def get_model_features(pipe: Pipeline, step_name: str) -> list:

    features = pipe.named_steps[step_name].get_feature_names_out()
    try:
        transformed_features_list = [feature.split('__')[1] for feature in features]
    except:
        pass
    
    return transformed_features_list

In [ ]:
def get_shap_values(pipe, X_train, X_test):

    logging.getLogger('shap').setLevel(logging.WARNING)

    estimator = pipe.estimator

    model_name = get_model_name(estimator)

    # preprocess the data
    X_train_transformed = estimator.named_steps["pre_processing"].transform(X_train)
    X_test_transformed = estimator.named_steps["pre_processing"].transform(X_test)

    # get feature names
    transformed_features = get_model_features(estimator, "pre_processing")

    X_test_transformed = pd.DataFrame(
        data=X_test_transformed, columns=transformed_features
    )

    # extract the model from the pipeline
    ml_model = estimator.named_steps[model_name]

    # check if the model has a predict_proba method
    if not hasattr(ml_model, "predict_proba"):
        msg = (f"{model_name} does not "
               f"support SHAP explanations. SHAP plot not created.")
        print(msg)
        raise Exception(msg)

    # initialize SHAP explainer
    if model_name=='Logistic Regression':
        masker = shap.maskers.Impute(X_train_transformed)
        explainer = shap.LinearExplainer(ml_model, masker=masker)
    else:
        explainer = shap.TreeExplainer(ml_model)
        
    shap_values = explainer.shap_values(X_test_transformed)

    try:
        shap_values = shap_values[:,:,1]
    except:
        raise Exception("Shap values wrong shape")
        
    return shap_values

In [ ]:
def order_feature_importance(X_train, transformed_feature_names, shap_values):

    feature_base = []
    for transformed_feature in transformed_feature_names:
        marker = False
        for feature in X_train.columns:
            if transformed_feature==feature:
                feature_base.append(feature)
                marker = True
        if not marker:
            for feature in X_train.columns:
                if transformed_feature.startswith(feature):
                    feature_base.append(feature)
                    marker = True
    
    df_features = pd.DataFrame(data=[feature_base], columns=transformed_feature_names)
    df_shap_values = pd.DataFrame(data=shap_values, columns=transformed_feature_names)

    df = pd.concat([df_features, df_shap_values], axis=0)

    base_avg_shap_values = []
    tran_avg_shap_values = []
    base_cols = []
    tran_cols = []
    for col_idx, col in enumerate(df.columns):
        tran_avg = df.iloc[1:, col_idx].abs().mean()
        tran_avg_shap_values.append(tran_avg)
        tran_cols.append(col)

        base_col = df.iloc[0, col_idx]
        if base_col not in base_cols:
            base_cols.append(base_col)
            base_feature_idxs = []
            for i, feature in enumerate(feature_base):
                if base_col==feature:
                    base_feature_idxs.append(i)
            base_avg = np.mean(np.abs(df.iloc[1:, base_feature_idxs].values))
            base_avg_shap_values.append(base_avg)
    
    df_base = pd.DataFrame({'feature': base_cols,
                            'mean_shap_value': base_avg_shap_values})
    df_tran = pd.DataFrame({'transformed_feature': tran_cols,
                            'base_feature': feature_base,
                            'mean_shap_value': tran_avg_shap_values})
    
    df_base = df_base.sort_values(by='mean_shap_value', ascending=False).reset_index(drop=True)
    df_tran = df_tran.sort_values(by='mean_shap_value', ascending=False).reset_index(drop=True)

    return df_base, df_tran


In [ ]:
def create_preprocessing_pipeline(X_train, num_cols, disc_cols, cat_cols, removed_features=None):

    if removed_features is not None:
        num_cols = [col for col in num_cols if col not in removed_features]
        disc_cols = [col for col in disc_cols if col not in removed_features]
        cat_cols = [col for col in cat_cols if col not in removed_features]
        assert len(X_train.columns)==len(num_cols)+len(disc_cols)+len(cat_cols)
    
    invalid_features = list(
        set(num_cols+disc_cols+cat_cols) - set(X_train.columns.values.tolist())
    )
    if len(invalid_features) != 0:
        msg = f"The following features are not in the dataframe: {invalid_features}"
        print(msg)
        raise ValueError(msg)
    
    impute_and_scale = Pipeline([
        ("numeric_impute", SimpleImputer(strategy="median")),
        ("numeric_transformation", StandardScaler())
    ])
    binary_and_discrete_impute = Pipeline([
        ("numeric_impute", SimpleImputer(strategy="median"))
    ])
    impute_and_one_hot_encode = Pipeline([
        ("categorical_transformation", OneHotEncoder(handle_unknown='infrequent_if_exist'))
    ])

    transformers = []
    if len(num_cols)>0:
        transformers.append(
            ("numeric_preprocessing", impute_and_scale, num_cols)
        )
    if len(disc_cols)>0:
        transformers.append(
            ("binary_and_discrete_preprocessing", binary_and_discrete_impute, disc_cols)
        )
    if len(cat_cols)>0:
        transformers.append(
            ("categorical_preprocessing", impute_and_one_hot_encode, cat_cols)
        )

    if len(transformers)>0:
        return ColumnTransformer(transformers=transformers)
    else:
        raise Exception("There are no transformers")

In [ ]:
def retrain_model(model, X_train, y_train, num_cols, disc_cols, cat_cols, drop_cols):

    preprocessor = create_preprocessing_pipeline(X_train, num_cols, disc_cols, cat_cols, drop_cols)

    model_params = model.named_steps['rfc_model'].get_params()
    
    clf = RandomForestClassifier(**model_params)
    clf = model.named_steps['rfc_model']

    pipe = Pipeline(steps=[
        ('pre_processing', preprocessor),
        ('rfc_model', clf)
    ])

    pipe.fit(X_train, y_train)

    return pipe

In [ ]:
def predict_probabilitess(pipe, X):
    return pipe.predict_proba(X)[:, 1]

In [ ]:
def predict_outcomes(pipe, X, threshold):
    y_prob = predict_probabilitess(pipe, X)
    y_pred = y_prob >= threshold
    return y_pred

In [ ]:
def stratified_bootstrap_metrics(model, X, y, threshold, n_iterations=1000):
    roc_auc_scores = []
    pr_auc_scores = []
    f1_scores = []
    g_mean_scores = []
    mcc_scores = []

    y = np.array(y)

    class_0_indices = np.where(y==0)[0]
    class_1_indices = np.where(y==1)[0]

    n_class_0 = len(class_0_indices)
    n_class_1 = len(class_1_indices)

    for _ in range(n_iterations):
        
        resampled_class_0 = resample(class_0_indices, replace=True, n_samples=n_class_0)
        resampled_class_1 = resample(class_1_indices, replace=True, n_samples=n_class_1)

        resampled_indices = np.concatenate([resampled_class_0, resampled_class_1])
        np.random.shuffle(resampled_indices)

        X_resampled = X.iloc[resampled_indices, :]
        y_resampled = y[resampled_indices]

        y_pred = predict_outcomes(model, X_resampled, threshold)
        y_prob = predict_probabilitess(model, X_resampled)

        roc_auc = roc_auc_score(y_resampled, y_prob)
        precision, recall, _ = precision_recall_curve(y_resampled, y_prob)
        pr_auc = auc(recall, precision)
        f1 = f1_score(y_resampled, y_pred)
        g_mean = geometric_mean_score(y_resampled, y_pred)
        mcc = matthews_corrcoef(y_resampled, y_pred)

        roc_auc_scores.append(roc_auc)
        pr_auc_scores.append(pr_auc)
        f1_scores.append(f1)
        g_mean_scores.append(g_mean)
        mcc_scores.append(mcc)

    roc_auc_stats = (
        np.percentile(roc_auc_scores, 25),
        np.percentile(roc_auc_scores, 50),
        np.percentile(roc_auc_scores, 75)
    )
    pr_auc_stats = (
        np.percentile(pr_auc_scores, 25),
        np.percentile(pr_auc_scores, 50),
        np.percentile(pr_auc_scores, 75)
    )
    f1_stats = (
        np.percentile(f1_scores, 25),
        np.percentile(f1_scores, 50),
        np.percentile(f1_scores, 75)
    )
    g_mean_stats = (
        np.percentile(g_mean_scores, 25),
        np.percentile(g_mean_scores, 50),
        np.percentile(g_mean_scores, 75)
    )
    mcc_stats = (
        np.percentile(mcc_scores, 25),
        np.percentile(mcc_scores, 50),
        np.percentile(mcc_scores, 75)
    )

    return roc_auc_stats, pr_auc_stats, f1_stats, g_mean_stats, mcc_stats

## Settings

In [ ]:
model_filepath = 'models/rfc_model_full.pkl'
train_data_filepath = 'data/model_data/training_data_full.csv'
val_data_filepath = 'data/model_data/validation_data_full.csv'
test_data_filepath = 'data/model_data/testing_data_full.csv'

In [ ]:
num_cols = ['acute_morbidity_indicator', 'ae_duration_hrs', 'max_tnt_24hr_int',
            'min_egfr_24hr_int', 'first_tnt_24hr_int', 'first_egfr_24hr_int',
            'mood_and_anxiety_disorders_indicator', 'tnt_egfr_interaction',
            'ip_duration_days', 'total_duration_days', 'age', 'tnt_change', 'egfr_change']
disc_cols = ['ihd_mi', 'cc_heart_failure', 'cc_myocardial_infarction',
             'imd_decile_19', 'qof_diabetes', 'qof_ht', 'ht', 'qof_chd',
             'ihd_nonmi', 'af', 'arrhythmia_other', 'stroke', 'hf', 'vasc_dis',
             'cardio_other', 'qof_depression', 'qof_mental', 'N_tnt_24hr', 'N_egfr_24hr',
             'mi_diagnosis_ae_discharge', 'meds_total', 'meds_antip', 'meds_angio',
             'meds_betab', 'meds_total_discharge', 'transfered_dv', 'mi_diagnosis_code',
             'chd_diagnosis_code', 'meds_total_more_than_10',
             'tnt_rule_in', 'age_threshold', 'ae_target', 'egfr_rule_in']
cat_cols = ['ethnicity', 'sex', 'smoking', 'ae_provider', 'ip_provider',
            'site_ae', 'site_ip', 'derived_trust_catchment',
            'departure_season', 'diagnosis_description']

## Analysis

### Load model and data

In [ ]:
rfc_model = load_model(model_filepath)
threshold = rfc_model.best_threshold_
print(threshold)

In [ ]:
train_data = load_csv(train_data_filepath)
val_data = load_csv(val_data_filepath)
test_data = load_csv(test_data_filepath)

In [ ]:
X_train = train_data.drop(['nhs_number', 'subsequent_mi_30days_diagnosis'], axis=1).copy()
y_train = train_data['subsequent_mi_30days_diagnosis'].copy()

In [ ]:
X_val = val_data.drop(['nhs_number', 'subsequent_mi_30days_diagnosis'], axis=1).copy()
y_val = val_data['subsequent_mi_30days_diagnosis'].copy()

In [ ]:
X_test = test_data.drop(['nhs_number', 'subsequent_mi_30days_diagnosis'], axis=1).copy()
y_test = test_data['subsequent_mi_30days_diagnosis'].copy()

In [ ]:
for col in disc_cols:
    X_train[col] = X_train[col].astype('category')
    X_val[col] = X_val[col].astype('category')
    X_test[col] = X_test[col].astype('category')

In [ ]:
y_pred = predict_outcomes(rfc_model, X_test, threshold)
y_prob = predict_probabilitess(rfc_model, X_test)

roc_auc = roc_auc_score(y_test, y_prob)
precision, recall, _ = precision_recall_curve(y_test, y_prob)
pr_auc = auc(recall, precision)
f1 = f1_score(y_test, y_pred)
g_mean = geometric_mean_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)

main_model_metrics = [roc_auc, pr_auc, f1, g_mean, mcc]

print(round(roc_auc, 3))
print(round(pr_auc, 3))
print(round(f1, 3))
print(round(g_mean, 3))
print(round(mcc, 3))

### Order feature importance

In [ ]:
shap_values = get_shap_values(rfc_model, X_train, X_test)

In [ ]:
transformed_feature_names = get_model_features(rfc_model.estimator, 'pre_processing')

In [ ]:
df_base, df_tran = order_feature_importance(X_train, transformed_feature_names, shap_values)

In [ ]:
print(len(df_base), len(df_tran))

### Retrain models and store evaluation metrics

In [ ]:
m = retrain_model(rfc_model.estimator, X_train, y_train, num_cols, disc_cols, cat_cols, [])
y_pred = predict_outcomes(m, X_test, threshold)
y_prob = predict_probabilitess(m, X_test)

roc_auc = roc_auc_score(y_test, y_prob)
precision, recall, _ = precision_recall_curve(y_test, y_prob)
pr_auc = auc(recall, precision)
f1 = f1_score(y_test, y_pred)
g_mean = geometric_mean_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)

print(round(roc_auc, 3))
print(round(pr_auc, 3))
print(round(f1, 3))
print(round(g_mean, 3))
print(round(mcc, 3))

In [ ]:
print(train_data.shape, X_train.shape, y_train.shape)
print(val_data.shape, X_val.shape, y_val.shape)
print(test_data.shape, X_test.shape, y_test.shape)

In [ ]:
rfc_model.best_threshold_

In [ ]:
feature_list = []
roc_auc_list = []
pr_auc_list = []
f1_score_list = []
g_mean_list = []
mcc_list = []

for i, col in enumerate(df_base['feature']):
    
    feature_list.append(col)
    drop_cols = list(set(X_train.columns)-set(feature_list))

    X_train_temp = X_train[feature_list].copy()
    X_test_temp = X_test[feature_list].copy()
    
    new_model = retrain_model(rfc_model.estimator, X_train_temp, y_train,
                              num_cols, disc_cols, cat_cols, drop_cols)
    
    stats = stratified_bootstrap_metrics(new_model, X_test_temp, y_test, threshold, 500)

    roc_auc_list.append(stats[0])
    pr_auc_list.append(stats[1])
    f1_score_list.append(stats[2])
    g_mean_list.append(stats[3])
    mcc_list.append(stats[4])

analysis_df = pd.DataFrame({
    'Number of Features': range(1, len(feature_list)+1),
    'Features': feature_list,
    'ROC-AUC': roc_auc_list,
    'PR-AUC': pr_auc_list,
    'F1-Score': f1_score_list,
    'G-Mean': g_mean_list,
    'MCC': mcc_list
})

### Plot results

In [ ]:
analysis_df.tail()

In [ ]:
fig_letters = ['(a)', '(b)', '(c)', '(d)', '(e)']

fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(12, 16))

for letter, metric, benchmark, ax in zip(fig_letters, analysis_df.columns.tolist()[2:], main_model_metrics, axs.ravel()):
    
    ax.axhline(y=benchmark, linestyle='--', color='black', label=f'Main Model {metric} ({benchmark:.3f})')

    medians = []
    lqs = []
    uqs = []
    for i in range(len(analysis_df)):
        lqs.append(analysis_df.loc[i, metric][0])
        medians.append(analysis_df.loc[i, metric][1])
        uqs.append(analysis_df.loc[i, metric][2])

    ax.plot(analysis_df['Number of Features'], medians, label=f"Retrained Model Median {metric}", color='blue')
    ax.fill_between(analysis_df['Number of Features'], lqs, uqs, color='blue', alpha=0.2, label="IQR (25th-75th percentile)")

    ax.set(title=f"{letter} {metric} values",
           xlabel="Number of Features",
           ylabel=metric)
    
    ax.legend(loc='lower right')

    ax.grid()

ax5 = axs.ravel()[-1]
ax5.set_visible(False)

fig.suptitle("Feature Importance Analysis", fontsize=18)

plt.tight_layout(pad=3.0)
plt.savefig('results/supplementary_results/feature_importance_analysis.png')
plt.show()
plt.close()